[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/mifumo081a/pytorch_template/blob/master/example_mnist.ipynb)

In [6]:
!git clone https://github.com/mifumo081a/pytorch_template.git
!ls
%cd pytorch_template

fatal: destination path 'pytorch_template' already exists and is not an empty directory.
drive  pytorch_template  sample_data
/content/pytorch_template


In [7]:
!pip install -r pytorch_template/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from torchvision import transforms, datasets, models
import os
import torchinfo

In [16]:
import pytorch_template.cross_varidation as cv
from pytorch_template.utils import *
from pytorch_template.trainer import Trainer_Classifier

ModuleNotFoundError: ignored

In [10]:
root = os.getcwd()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TRAIN =True

In [11]:
logs_root = os.path.join(root, "logs/", "mnist/")
os.makedirs(logs_root, exist_ok=True)

In [12]:
trainval_dataset = datasets.MNIST(os.path.join(root, "data/"), train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(os.path.join(root, "data/"), train=False, download=True, transform=transforms.ToTensor())

trainval_dataset.labels = [str(i) for i in range(10)]
test_dataset.labels  = [str(i) for i in range(10)]

transforms_dict = {"train": 
                   transforms.Compose([
                       transforms.ToTensor()
                       ]),
                   "val":
                    transforms.Compose([
                       transforms.ToTensor()
                       ]),
                   "test":
                   transforms.Compose([
                       transforms.ToTensor()
                       ])
}

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /content/pytorch_template/data/MNIST/raw/train-images-idx3-ubyte.gz to /content/pytorch_template/data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /content/pytorch_template/data/MNIST/raw/train-labels-idx1-ubyte.gz to /content/pytorch_template/data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /content/pytorch_template/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/pytorch_template/data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /content/pytorch_template/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/pytorch_template/data/MNIST/raw



In [13]:
num_workers = os.cpu_count()
n_splits = 5
batch_size = 50
if TRAIN:
    kfold_root = cv.kfold_split(dataset=trainval_dataset, test_dataset=test_dataset, transforms=transforms_dict, n_splits=n_splits,
                                root=root, num_workers=num_workers, batch_size=batch_size)
else:
    os.path.join(root, "kfold_pickle")
print(kfold_root)

NameError: ignored

# Model initialize and Train

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = models.vgg16(pretrained=True).features[:10]
        for layer in self.features:
            for param in layer.parameters():
                param.requires_grad = False
        # self.avgpool = models.vgg16().avgpool
        self.classifier = nn.Sequential(
            nn.Linear(128, 10),
            nn.ReLU(),
        )
        
    def forward(self,x):
        x = x.repeat(1, 3, 1, 1)
        fmaps = self.features(x)
        # global average pooling
        x = fmaps.mean([2,3])
        x = self.classifier(x)
        
        return x, fmaps

torchinfo.summary(Model(), (1, 1, 28, 28))

In [ ]:
model_root = os.path.join(root, "models/")
print(model_root)
if TRAIN:
    set_model(model=Model(), model_root=model_root)

In [14]:
if TRAIN:
  trainer_list = []
  for k in range(n_splits):
    path = os.path.join(kfold_root, str(k)+"/")
    trainer_list.append(
        Trainer_Classifier(model=get_model("nontrain", model_root, device),
                           device=device,
                           dataloaders=get_dataloaders(path),
                           epochs=10)
    )

NameError: ignored

In [ ]:
if TRAIN:
    cv.kfold_train(save_model_root=model_root, n_splits=n_splits,
                   trainer_list=trainer_list)

In [15]:
if TRAIN:
    for k in range(n_splits):
        path = os.path.join(kfold_root, str(k)+"/")
        trainer_list[k].show_curve(logs_root=os.path.join
                                   (logs_root, "curves/"), fname=str(k), save=False)

NameError: ignored

# Evaluate models

In [ ]:
from pytorch_template.evaluator import ImageClassifier_Evaluator

In [ ]:
evaluator_list = []
for k in range(n_splits):
    dataloader_path = os.path.join(kfold_root, str(k)+"/")
    model_path = os.path.join(model_root, str(k)+"/")
    print(dataloader_path, model_path)
    evaluator_list.append(ImageClassifier_Evaluator(model=get_model("model_fit", model_path, device),
                                          device=device,
                                          dataloaders=get_dataloaders(dataloader_path),
                                          testloaders=get_testloaders(kfold_root),
                                          logs_root=logs_root
                                         )
                         )

In [ ]:
kfold_acc = []
for k in range(n_splits):
    evaluator_list[k].confusion_matrix(folder_name="confusion_matrix", fname=str(k), save=True)
    kfold_acc.append(evaluator_list[k].acc)

In [ ]:
print("Acc mean: {:.4f}, std: {:.4f}".format(np.array(kfold_acc).mean(), np.array(kfold_acc).std()))

In [ ]:
for k in range(n_splits):
    evaluator_list[k].show_scores(folder_name="eval", fname=str(k), save=True)

In [ ]:
for k in range(n_splits):
    evaluator_list[k].show_cam(folder_name="cam", fname=str(k), save=True)